# Check Token Usage

Hướng dẫn này bao gồm cách theo dõi và giám sát việc sử dụng token với `LangChain` và `OpenAI API`.

`Token usage tracking` (theo dõi việc sử dụng token) rất quan trọng để quản lý chi phí API và tối ưu hóa việc sử dụng tài nguyên.

Trong hướng dẫn này, chúng ta sẽ tạo một ví dụ đơn giản để đo lường và giám sát mức tiêu thụ token trong các lệnh gọi `OpenAI API` bằng cách sử dụng `CallbackHandler` của LangChain.

![](https://raw.githubusercontent.com/aidino/LangChain-OpenTutorial/683d91eb3fd1028345ee8e5b80660b3b0fcc6b96/04-Model/assets/04-CheckTokenUsage-example-flow-token-usage.png)


In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
prompt = PromptTemplate.from_template("Liệt kê 5 món ăn ngon ở {place}")
chain = prompt | llm | StrOutputParser()

# Create cache directory
if not os.path.exists("cache"):
    os.makedirs("cache")

# Set SQLiteCache
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

answer = chain.invoke("Hà Nội")
print(answer)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và hấp dẫn.

2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.

3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate có vị ngon, thơm và béo của pate kết hợp với vị giòn của bánh mỳ.

4. Nem chua rán: Một món ăn vặt phổ biến của người dân Hà Nội, nem chua rán có vị chua, cay, ngọt và giòn rất hấp dẫn.

5. Bún ốc: Một món ăn đặc trưng của Hà Nội, bún ốc có hương vị đậm đà, thơm ngon và hấp dẫn, được nhiều người yêu thích.


In [2]:
chain.invoke("Hà Nội")

'1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và hấp dẫn.\n\n2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.\n\n3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate có vị ngon, thơm và béo của pate kết hợp với vị giòn của bánh mỳ.\n\n4. Nem chua rán: Một món ăn vặt phổ biến của người dân Hà Nội, nem chua rán có vị chua, cay, ngọt và giòn rất hấp dẫn.\n\n5. Bún ốc: Một món ăn đặc trưng của Hà Nội, bún ốc có hương vị đậm đà, thơm ngon và hấp dẫn, được nhiều người yêu thích.'

## Implementing Check Token Usage

if you want to check token usage, you can use `get_openai_callback` function.

In [3]:
from langchain_community.callbacks.manager import get_openai_callback

with get_openai_callback() as cb:
  result = chain.invoke("Hà Nội")
  print(cb)

Tokens Used: 389
	Prompt Tokens: 28
		Prompt Tokens Cached: 0
	Completion Tokens: 361
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0005555


In [4]:
# callback to track it
with get_openai_callback() as cb:
    result = llm.invoke("where is the capital of United States?")
    print(f"Total tokens used: \t\t{cb.total_tokens}")
    print(f"Tokens used in prompt: \t\t{cb.prompt_tokens}")
    print(f"Tokens used in completion: \t{cb.completion_tokens}")
    print(f"Cost: \t\t\t\t${cb.total_cost}")

Total tokens used: 		28
Tokens used in prompt: 		15
Tokens used in completion: 	13
Cost: 				$2.7e-05


## Monitoring Token Usage

Việc theo dõi sử dụng token (Token usage monitoring) là rất quan trọng để quản lý chi phí và tài nguyên khi sử dụng OpenAI API. LangChain cung cấp một cách dễ dàng để theo dõi điều này thông qua `get_openai_callback()`.

Trong phần này, chúng ta sẽ khám phá việc theo dõi sử dụng token thông qua ba kịch bản chính:

1.  **Theo dõi truy vấn đơn (Single Query Monitoring)**:
    -   Theo dõi sử dụng token cho các lệnh gọi API riêng lẻ (individual API calls)
    -   Phân biệt giữa token prompt và token completion
    -   Tính toán chi phí (calculate costs)

2.  **Theo dõi nhiều truy vấn (Multiple Queries Monitoring)**:
    -   Theo dõi sử dụng token tích lũy (cumulative token usage) trên nhiều lệnh gọi API
    -   Phân tích tổng chi phí (analyze total costs)

>   **Lưu ý (Note)**: Việc theo dõi sử dụng token hiện chỉ được hỗ trợ cho OpenAI API.


In [5]:
# 1. Single Query Monitoring
print("1. Single Query Monitoring")
print("-" * 40)

with get_openai_callback() as cb:
    response = llm.invoke("What is the capital of France?")
    print(f"Response: {response.content}")
    print("-" * 40)
    print(f"Token Usage Statistics:")
    print(f"Prompt Tokens: \t\t{cb.prompt_tokens}")
    print(f"Completion Tokens: \t{cb.completion_tokens}")
    print(f"Total Tokens: \t\t{cb.total_tokens}")
    print(f"Cost: \t\t\t${cb.total_cost:.4f}\n")

1. Single Query Monitoring
----------------------------------------
Response: The capital of France is Paris.
----------------------------------------
Token Usage Statistics:
Prompt Tokens: 		14
Completion Tokens: 	8
Total Tokens: 		22
Cost: 			$0.0000



In [6]:
# 2. Multiple Queries Monitoring
print("2. Multiple Queries Monitoring")
print("-" * 40)

with get_openai_callback() as cb:
    # First query
    response1 = llm.invoke("What is Python?")
    # Second query
    response2 = llm.invoke("What is JavaScript?")

    print(f"Response 1: {response1.content[:100]}...")
    print("-" * 40)
    print(f"Response 2: {response2.content[:100]}...")
    print("-" * 40)
    print("Cumulative Statistics:")
    print(f"Total Prompt Tokens: \t\t{cb.prompt_tokens}")
    print(f"Total Completion Tokens: \t{cb.completion_tokens}")
    print(f"Total Tokens: \t\t\t{cb.total_tokens}")
    print(f"Total Cost: \t\t\t${cb.total_cost:.4f}\n")

2. Multiple Queries Monitoring
----------------------------------------
Response 1: Python is a high-level, interpreted programming language known for its simplicity and readability. I...
----------------------------------------
Response 2: JavaScript is a high-level, dynamic, and interpreted programming language that is commonly used for ...
----------------------------------------
Cumulative Statistics:
Total Prompt Tokens: 		22
Total Completion Tokens: 	122
Total Tokens: 			144
Total Cost: 			$0.0002

